In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

# Define relevant variables for the ML task
batch_size = 32
num_classes = 10
learning_rate = 0.001
num_epochs = 1

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.CNN1=nn.Conv2d(3,64,kernel_size=(2,2),padding='same')
        self.CNN2=nn.Conv2d(64,64,kernel_size=(2,2),padding='same')
        self.CNN3=nn.Conv2d(64,64,kernel_size=(2,2),padding='same')
        self.CNN4=nn.Conv2d(64,64,kernel_size=(2,2),padding='same')
        self.CNN5=nn.Conv2d(64,64,kernel_size=(2,2),padding='same')
        self.CNN6=nn.Conv2d(64,64,kernel_size=(2,2),padding='same')
        self.MP=nn.MaxPool2d(kernel_size=2)
        self.fc1=nn.Linear(4096,1024)
        self.fc2=nn.Linear(1024,num_classes)
        
    
    # Progresses data across layers    
    def forward(self, x):
        #print(x.shape)
        x1=self.CNN1(x)
        x1=F.relu(x1)
        x2=self.CNN2(x1)
        x2=F.relu(x2)
        x3=self.CNN3(x2)
        x3=F.relu(x3)
        x4=self.CNN4(x3)
        x4=F.relu(x4)
        x4=self.MP(x4)
        x5=self.CNN5(x4)
        x5=F.relu(x5)
        x6=self.CNN6(x5)
        x6=F.relu(x6)
        x7=self.MP(x6)
        x8=torch.flatten(x7, start_dim=1)
        x9=self.fc1(x8)
        x10=self.fc2(x9)
        return x10

In [16]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

In [17]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
import time
start = time.time()
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
end = time.time()
print("The time of execution of above program is :", end-start)           
            


KeyboardInterrupt: 

In [ ]:
import time
start = time.time()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))
end = time.time()
print("The time of execution of above program is :", end-start)

